In [ ]:
import os, pickle
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA

import manifold_dynamics.tuning_utils as tut

In [ ]:
%matplotlib inline

In [ ]:
DATA_DIR = '../../datasets/NNN/'
IMG_DIR = '../../datasets/NNN/NSD1000_LOC'

CAT = 'face'
dat = pd.read_pickle(os.path.join(DATA_DIR, (f'{CAT}_roi_data.pkl')))
    ROI_LIST = list(dat['roi'].unique())
print(f'Unique face ROIs: {ROI_LIST}')

with open(f'../../datasets/NNN/{CAT}_mins.pkl', 'rb') as f:
    mins = pickle.load(f)
    
SCALES = {k: v[0] for k,v in mins.items()}

In [ ]:
ROI = 'MF1_9_F'
tstart = 100
tend=350
scale = SCALES[ROI]

# find top images
rng = np.random.default_rng(0)
scores = tut.landscape(dat, ROI)
order = np.argsort(scores)[::-1]
idx = order[:scale]

R, _ = tut.specific_static_rdm(dat, ROI, idx, tstart=tstart, tend=tend)
sns.heatmap(R, square=True, vmin=0, vmax=1)

pc_input = []
for t in range(100, 300, 2):
    v = tut.idkman(dat, ROI, window=slice(t, t+50), images=idx)
    pc_input.append(v)
pc_input = np.array(pc_input)
print(pc_input.shape)

In [ ]:
def compute_fixed_pca_trajectories(X, n_components=10, center=True):
    '''
    X: array (time, units, images)
    returns:
        traj: (time, images, n_components)
        pca: fitted PCA object
    '''
    T, U, I = X.shape
    
    # reshape to (samples, units)
    # samples = time × image
    X_reshaped = X.transpose(0, 2, 1).reshape(T * I, U)
    
    # center across samples (important)
    if center:
        X_mean = np.nanmean(X_reshaped, axis=0, keepdims=True)
        X_reshaped = X_reshaped - X_mean
    else:
        X_mean = np.zeros((1, U))
    
    # fit once
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_reshaped)
    
    # reshape back to (time, images, components)
    traj = X_pca.reshape(T, I, n_components)
    
    return traj, pca, X_mean

In [ ]:
traj, pca, X_mean = compute_fixed_pca_trajectories(pc_input, n_components=2)
print(traj.shape)

T, I, D = traj.shape

time_idx = np.repeat(np.arange(T), I)
image_idx = np.tile(np.arange(I), T)

df = pd.DataFrame({
    'time': time_idx,
    'image': image_idx,
    # 'set': I // image_idx,      # each set of 30 images shares a label
    'pc1': traj[:, :, 0].reshape(-1),
    'pc2': traj[:, :, 1].reshape(-1),
})

# customp = sns.color_palette('Greys')
fig, ax = plt.subplots(1, 1, figsize=(8,5))

# thin trajectory lines per image
# sns.lineplot(data=df, x='pc1', y='pc2', units='image',estimator=None,color='k',linewidth=0.5, alpha=0.3,ax=ax)

# scatter on top
sns.scatterplot(
    data=df,
    x='pc1',
    y='pc2',
    hue='time',
    # alpha = 0.5,
    s=25,
    linewidth=0,
    ax=ax
)

ax.legend().remove()
ax.set_aspect('equal', adjustable='datalim')
ax.set_title(f'{ROI} local scale trajectories')

In [ ]:
traj, pca, X_mean = compute_fixed_pca_trajectories(pc_input, n_components=3)
# traj = traj[:, [10], :]

print(traj.shape)

T, I, D = traj.shape

time_idx = np.repeat(np.arange(T), I)
image_idx = np.tile(np.arange(I), T)

df = pd.DataFrame({
    'time': time_idx,
    'image': image_idx,
    # 'set': I // image_idx,      # each set of 30 images shares a label
    'pc1': traj[:, :, 0].reshape(-1),
    'pc2': traj[:, :, 1].reshape(-1),
    'pc3': traj[:, :, 2].reshape(-1),
})

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df['pc1'],df['pc2'],df['pc3'], c=np.arange(len(df)), s=20, cmap=sns.cubehelix_palette(as_cmap=True))
# ax.plot(df['pc1'],df['pc2'],df['pc3'], linewidth=0.1, color='black')

ax.set_title(f'{ROI} local scale trajectories')
angle=0
ax.view_init(0, angle)
plt.show()

In [ ]:
tstart = 100
tend=350

# find top images
rng = np.random.default_rng(0)
scores = tut.landscape(dat, ROI)
order = np.argsort(scores)[::-1]
idx = order[-30:]#rng.choice(order[scale:], scale)

R, _ = tut.specific_static_rdm(dat, ROI, idx, tstart=tstart, tend=tend)
sns.heatmap(R, square=True, vmin=0, vmax=1)

pc_input = []
for t in range(100, 300, 2):
    v = tut.idkman(dat, ROI, window=slice(t, t+50), images=idx)
    pc_input.append(v)
pc_input = np.array(pc_input)
print(pc_input.shape)

In [ ]:
traj, pca, X_mean = compute_fixed_pca_trajectories(pc_input, n_components=2)
print(traj.shape)

T, I, D = traj.shape

time_idx = np.repeat(np.arange(T), I)
image_idx = np.tile(np.arange(I), T)

df = pd.DataFrame({
    'time': time_idx,
    'image': image_idx,
    # 'set': I // image_idx,      # each set of 30 images shares a label
    'pc1': traj[:, :, 0].reshape(-1),
    'pc2': traj[:, :, 1].reshape(-1),
})

# customp = sns.color_palette('Greys')
fig, ax = plt.subplots(1, 1, figsize=(8,5))

# thin trajectory lines per image
# sns.lineplot(data=df, x='pc1', y='pc2', units='image',estimator=None,color='k',linewidth=0.5, alpha=0.3,ax=ax)

# scatter on top
sns.scatterplot(
    data=df,
    x='pc1',
    y='pc2',
    hue='time',
    s=25,
    linewidth=0,
    ax=ax
)

ax.legend().remove()
ax.set_aspect('equal', adjustable='datalim')
ax.set_title(f'{ROI} random scale trajectories')

In [ ]:
traj, pca, X_mean = compute_fixed_pca_trajectories(pc_input, n_components=3)
print(traj.shape)

T, I, D = traj.shape

time_idx = np.repeat(np.arange(T), I)
image_idx = np.tile(np.arange(I), T)

df = pd.DataFrame({
    'time': time_idx,
    'image': image_idx,
    # 'set': I // image_idx,      # each set of 30 images shares a label
    'pc1': traj[:, :, 0].reshape(-1),
    'pc2': traj[:, :, 1].reshape(-1),
    'pc3': traj[:, :, 2].reshape(-1),
})

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df['pc1'],df['pc2'],df['pc3'], c=np.arange(len(df)), s=20, cmap=sns.cubehelix_palette(as_cmap=True))
ax.plot(df['pc1'],df['pc2'],df['pc3'], linewidth=0.1, color='black')

ax.set_title(f'{ROI} random scale trajectories')
angle=45
ax.view_init(0, angle)
plt.show()